## Neural Network Embedding
> [참고](https://keraskorea.github.io/posts/2018-10-16-%EB%89%B4%EB%9F%B4%20%EB%84%A4%ED%8A%B8%EC%9B%8C%ED%81%AC%20%EC%9E%84%EB%B2%A0%EB%94%A9%EC%9D%84%20%EC%9D%B4%EC%9A%A9%ED%95%9C%20%EC%B6%94%EC%B2%9C%20%EC%8B%9C%EC%8A%A4%ED%85%9C%20%EB%A7%8C%EB%93%A4%EA%B8%B0/)

- 유사한 위키피디아 페이지로 연결되는 책들은 서로 비슷하다 > 위키피디아 링크를 기준으로 nearest neighbor을 그룹핑할 수 있다 > 링크 기준으로 유사도 계산
- Data: 첵의 제목, 기본정보, 위키링크, 외부 사이트 링크 > 제목+위키링크
    - 가장 많이 연결된 위키 페이지: paperback, hardcover 등은 책의 특징을 가리는데 영향을 주지 않는다->제거
    - (책 제목, 위키링크)가 주어지면 위키링크가 책의 기사에 존재하는지 확인
    - 책 이름에 고유 인덱스 부여
    
- 위키피디아 링크 = beer의 정보 (score, taste, smell 등)
- Data: beerID, beer정보(score,smell 등*확인필요*)
- BeerID는 이미 고유 인덱스 형태이므로 변환할 필요가 없다

1. input: beerID, score에 대한 병렬 입력
2. Embedding: beerID, score를 위한 병렬길이 50개의 임베딩 - 목표의 손실을 최소화하기 위한 신경망의 가중치(weight)
3. Dot: 내적을 계산하여 임베딩 합치기
4. Reshape: 임베딩 형태를 단일 숫자로
5. Dense: 시그모이드 활성화를 위한 출력

In [1]:
import pandas as pd
import numpy as np

In [19]:
data = pd.read_csv('my_df.csv')
data_new = data.drop('user',axis = 1)

In [16]:
# pairs 대신
data2 = data_new.values

array([4.97, 4.04, 3.71, ..., 4.05, 4.04, 4.26])

In [20]:
beer_index = data2[:,0]
rating = data2[:,1]
len(beer_index);len(rating)

101889

### (1) 임베딩 모델 구축하기

In [21]:
from keras.layers import Input, Embedding, Dot, Reshape, Dense
from keras.models import Model

def beer_embedding_model(embedding_size = 50, classification = False):
    # input
    beer = Input(name = 'beer', shape = [1])
    rating = Input(name = 'rating', shape = [1])
    
    # beer 임베딩
    beer_embedding = Embedding(name = 'beer_embedding',input_dim = len(beer_index), output_dim = embedding_size)(beer)
    # rating 임베딩
    rating_embedding = Embedding(name = 'rating_embedding',input_dim = len(rating),output_dim = embedding_size)(rating)
    
    # Dot(내적)으로 shape(None,1,1)로 줄이기
    merged = Dot(name = 'dot_product', normalize = True, axes = 2)([beer_embedding,rating_embedding])
    
    # 단일 숫자 shape(None,1)로 줄이기
    merged = Reshape(target_shape = [1])(merged)
    
    # output 출력
    out = Dense(1, activation = 'sigmoid')(merged)
    model = Model(inputs = [beer,rating], outputs = out)
    
    #학습 단계
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', matrics = ['accuracy'])
    
    return model

In [27]:
data2_list = data2.tolist
data2_list
#data_set = set(data2_list)

<function ndarray.tolist>

In [22]:
import numpy as np
import random
random.seed(100)

def generate_batch(data2, n_positive=50, negative_ratio = 1.0):
    # batch 저장용 array
    batch_size = n_positive*(1+negative_ratio)
    batch = np.zeros((batch_size,3))
    
    #샘플 준비
    while True:
        for idx, (beerID,rating_i) in enumerate(random.sample(data2,n_positive)):
            batch[idx,:] = (beerID,rating_i,1)
        idx += 1
        
        while idx < batch_size:
            random_beer = random.randrange(len(beerID))
            random_rating = random.randrange(len(rating))
            
            if (random_beer,random_rating) not in data2_set:
                batch[idx,:] = (random_beer,random_rating,neg_label)
                idx += 1
                
        np.random.shuffle(batch)
        yield{'beer':batch[:,0],'rating':batch[:,1]},batch[:,2]              

In [23]:
n_postive = 1024

gen = generate_batch(data2,n_postive,negative_ratio=2)

h = model.fit_generator(gen, epoch=15, steps_per_epoch = len(data2)//n_postive)

NameError: name 'model' is not defined